In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#read the dataset and store it in a variable
#describe and print the head of data 
data=pd.read_csv('../input/tweets-sentiment-classification/train_2kmZucJ.csv')
print(data.head)
data.describe()
type(data)

In [ ]:
#preproces the data(data-cleaning)
#remove all the duplicates and remove all the null values
#print the shape of the dataset
data.dropna()
data.drop_duplicates()
print(data.shape)

In [ ]:
#print columns of dataset
data.columns

In [ ]:
tweets=data['tweet'].values
y=data['label'].values
tweets,y

In [ ]:
pos=data[data['label']==0]
neg=data[data['label']==1]
print(pos.shape)
print(neg.shape)
pos.columns
type(pos)
pos.head()
neg.head()


In [ ]:
pos.describe()
neg.describe()

In [ ]:
import matplotlib.pyplot as plt
labels='positive','negative'
colors=['green','red']
sizes=[pos.shape[0],neg.shape[0]]
plt.pie(sizes,labels=labels,colors=colors,autopct='%1.1f%%',shadow=True)
plt.axis('equal')
plt.show()

In [ ]:
train_pos=pos[:4000]
test_pos=pos[4000:]
train_neg=neg[:1500]
test_neg=neg[1500:]
train_x=pd.concat([train_pos,train_neg])
test_x=pd.concat([test_pos,test_neg])

train_y = np.append(np.zeros(len(train_pos),dtype='int'), np.ones(len(train_neg),dtype='int'))
test_y = np.append(np.zeros(len(test_pos),dtype='int'), np.ones(len(test_neg),dtype='int'))
print(train_y.shape)
print(test_y.shape)
train_x=train_x['tweet']
type(train_x)
train_x=train_x.tolist()
test_x=test_x['tweet'].tolist()
type(train_x)
train_x[:10]
type(train_y)
print(len(train_x))
print(len(train_y))
train_y=train_y.tolist()



In [ ]:
import nltk
from nltk.corpus import stopwords
import string
from nltk.tokenize import TweetTokenizer
import matplotlib.pyplot as plt


In [ ]:
nltk.download('stopwords')

In [ ]:
import re                                  # library for regular expression operations
import string                              # for string operations
from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings

In [ ]:
def process_tweet(tweet):
    p_tweet = re.sub(r'^RT[\s]+', '', tweet)

    # remove hyperlinks
    p_tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', p_tweet)

    # remove hashtags
    # only removing the hash # sign from the word
    p_tweet = re.sub(r'#', '', p_tweet)
    #tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(p_tweet)
    
    #Import the english stop words list from NLTK
    stopwords_english = stopwords.words('english') 
    
    tweets_clean = []
    for word in tweet_tokens: # Go through every word in your tokens list
        if (word not in stopwords_english and word not in string.punctuation):  # remove punctuation
            tweets_clean.append(word)
    
    stemmer = PorterStemmer() 

    # Create an empty list to store the stems
    tweets_stem = [] 

    for word in tweets_clean:
        stem_word = stemmer.stem(word)  # stemming word
        tweets_stem.append(stem_word)  # append to the list
    return tweets_stem
    
   

In [ ]:
def count_tweets(result, tweets, ys):
    for y, tweet in zip(ys, tweets):
        for word in process_tweet(tweet):
            # define the key, which is the word and label tuple
            pair = (word,y)

            # if the key exists in the dictionary, increment the count
            if pair in result:
                result[pair] += 1

            # else, if the key is new, add it to the dictionary and set the count to 1
            else:
                result[pair] = 1
    ### END CODE HERE ###

    return result

In [ ]:
freqs=count_tweets({},train_x,train_y)
freqs

In [ ]:
def train_naive_bayes(freqs, train_x, train_y):
    loglikelihood = {}
    logprior = 0
    # calculate V, the number of unique words in the vocabulary
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)

    # calculate N_pos and N_neg
    N_pos = N_neg = V_pos =V_neg=0
    for pair in freqs.keys():

        if pair[1] == 1:
           
            V_neg+=1
            
            N_neg += freqs[pair]
    
        else:
            V_pos+=1
        
            N_pos += freqs[pair]


    # Calculate D, the number of documents
    D = len(pos)+len(neg)

    # Calculate D_pos, the number of positive documents
    D_pos = len(pos)

    # Calculate D_neg, the number of negative documents
    D_neg = len(neg)

    # Calculate logprior
    logprior = np.log(D_pos)-np.log(D_neg)

    # For each word in the vocabulary...
    for word in vocab:
        # get the positive and negative frequency of the word
        freq_pos = freqs.get(word,0)
        freq_neg = freqs.get(word,1)

        # calculate the probability that each word is positive, and negative
        p_w_pos = (freq_pos+1)/(N_pos+V)
        p_w_neg = (freq_neg+1)/(N_neg+V)

        # calculate the log likelihood of the word
        loglikelihood[word] = np.log(p_w_pos / p_w_neg)

    return logprior, loglikelihood


In [ ]:
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)
print(logprior)
print(len(loglikelihood))

In [ ]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
    '''
    Input:
        tweet: a string
        logprior: a number
        loglikelihood: a dictionary of words mapping to numbers
    Output:
        p: the sum of all the logliklihoods of each word in the tweet (if found in the dictionary) + logprior (a number)

    '''
    # process the tweet to get a list of words
    word_l = process_tweet(tweet)

    # initialize probability to zero
    p = 0

    # add the logprior
    p +=logprior

    for word in word_l:

        # check if the word exists in the loglikelihood dictionary
        if word in loglikelihood:
            # add the log likelihood of that word to the probability
            p += loglikelihood[word]

    return p


In [ ]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood):
    accuracy = 0
    y_hats = []
    for tweet in test_x:
        # if the prediction is > 0
        if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
            # the predicted class is 1
            y_hat_i = 1
        else:
            # otherwise the predicted class is 0
            y_hat_i =0

        # append the predicted class to the list y_hats
        y_hats.append(y_hat_i)

    # error is the average of the absolute values of the differences between y_hats and test_y
    error = np.mean(np.abs(y_hats-test_y))

    # Accuracy is 1 minus the error
    accuracy = 1-error

    ### END CODE HERE ###

    return accuracy


In [ ]:
print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))